In [1]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns
    
import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import dask.dataframe as dd

import gc
import sys

from multiprocessing import Pool
import multiprocessing

from dask.diagnostics import ProgressBar

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Get Data
Get the temperature data for all zipcodes here.

In [2]:
def getData(weatherType,yearRange1,yearRange2):

    if weatherType == "Tmax":
        weatherVar = "temperature"
    else: 
        weatherVar = "precipitation"
    
    year = yearRange1
    filename = "../../../../../../../Volumes/backup2/dissData/prism/zipcode" + weatherType + str(year) + ".csv"
    data = dd.read_csv(filename, assume_missing=True)[['ZIP','date',weatherVar]]
    data['ZIP'] = data.ZIP.astype('int64')

    
   
    years = range(yearRange1 + 1,yearRange2 + 1)
    for year in years:
        filename = "../../../../../../../Volumes/backup2/dissData/prism/zipcode" + weatherType + str(year) + ".csv"
        tempData = dd.read_csv(filename, assume_missing=True)[['ZIP','date',weatherVar]]
        tempData['ZIP'] = tempData.ZIP.astype('int64')

        # tempData = tempData[tempData.ZIP.isin(relevantZips)]
        # tempData = tempData[~(tempData[weatherVar].isna().compute())]
        data = data.append(tempData)

    data = data[~(data[weatherVar].isna())] # .compute()
    
    return(data)

In [39]:
def getPivotQuants(weatherType, identifiers,weatherData):
    start2 = time.time()
    quants_overallByZip   = weatherData.groupby(identifiers)[weatherType].quantile(q = quantiles).reset_index().\
        rename(columns=lambda x: re.sub('level_[0-9]$','quartile',x))
    # rename(columns = {'level_1': 'quartile'})
    
    print("got grouped")
    print(time.time() - start2)
    quants_overallByZip['quartile'] = 'q_' + quants_overallByZip['quartile'].astype(str).str.slice(0,6)

    ## 

    pivot_quants_overallByZip = quants_overallByZip.pivot(index=identifiers, 
            columns='quartile', values=str(weatherType)).reset_index().\
            rename_axis(None, axis=1)
    
    pivot_quants_overallByZip['q_0.0'] = -50
    pivot_quants_overallByZip['q_1.0'] = 1000
    # print(pivot_quants_overallByZip.head())

    
    # get the quartile list
    quartileList = pivot_quants_overallByZip.iloc[:,len(identifiers):].values.tolist()
    '''quartileList[0]  = -50
    quartileList[-1] = 1000'''
    
    pivot_quants_overallByZip['quartileList']      = quartileList
    pivot_quants_overallByZip['quartileLabelList'] = [list(pivot_quants_overallByZip.columns[2:-1])] * \
                                                                    pivot_quants_overallByZip.shape[0]

    pivot_quants_overallByZip = pivot_quants_overallByZip[pivot_quants_overallByZip.\
                                                            columns.drop(list(pivot_quants_overallByZip.filter(regex='q_')))]
    print(time.time() - start2)
    # pivot_quants_overallByZip.head()

    return(pivot_quants_overallByZip)

In [40]:
quantiles = [0.0, 0.50, 0.95, 1.0]

quant_labels = ['quant_tossThisOne', 'quant_50_95', 'quant_95_100']


## Run for temp and precip

In [7]:
weatherType = "Tmax"
weatherData = getData(weatherType, 1981, 1999)

print("got temp")


weatherType = "Precip" # Tmax
precipData = getData(weatherType, 1981, 1999)

print("got precip")

got temp
got precip


In [42]:
weatherData   = dd.read_csv("../../data/companyData/allZipsTemp_8199.csv").drop(columns = {'Unnamed: 0'})
precipData = dd.read_csv("../../data/companyData/allZipsPrecip_8199.csv").drop(columns = {'Unnamed: 0'})

with ProgressBar():
    precipData    = precipData.repartition(npartitions=225)
    weatherData   = weatherData.repartition(npartitions=225)

In [43]:
weatherData = weatherData.merge(precipData).compute(workers = 100)

print(weatherData.dtypes,sys.getsizeof(weatherData)/1e9)

weatherData.ZIP           = weatherData.ZIP.astype('int32')
weatherData.date          = weatherData.date.astype('int32')
weatherData.temperature   = weatherData.temperature.astype('float32')
weatherData.precipitation = weatherData.precipitation.astype('float32')


print(weatherData.dtypes)

sys.getsizeof(weatherData)/1e9

ZIP                int64
date             float64
temperature      float64
quarter            int64
precipitation    float64
dtype: object 10.876799264
ZIP                int32
date               int32
temperature      float32
quarter            int64
precipitation    float32
dtype: object


7.25119952

Get the week, month, quarter, and year for everything. The week is a little bit unusual, as it can depend on when Monday falls. But we want something a little bit different. It's referenced here: https://stackoverflow.com/questions/55889766/pandas-datetime-week-not-as-expected

In [44]:
weatherData['date'] = pd.to_datetime(weatherData['date'], format='%Y%m%d')
weatherData['dayOfYear'] = weatherData.date.dt.dayofyear
weatherData['week'] = weatherData.dayOfYear//7

In [45]:
weatherData.week.min()

0

In [46]:
weatherData['quarter']  = weatherData['date'].dt.quarter
weatherData['year']     = weatherData['date'].dt.year

In [47]:
weatherData.year.max()

1999

In [48]:
weatherData.year.drop_duplicates()

0         1998
27513     1999
80494     1981
127158    1982
180528    1983
238974    1984
292187    1985
344748    1986
397599    1987
450666    1988
503649    1989
556446    1990
609432    1991
662785    1992
716302    1993
769494    1994
815599    1995
866738    1996
928515    1997
Name: year, dtype: int64

In [49]:
weatherData.head()

,ZIP,date,temperature,quarter,precipitation,dayOfYear,week,year
0,70463,1998-05-08,31.167000,2,0.000,128,18,1998
1,70715,1998-05-08,30.639999,2,0.000,128,18,1998
2,71118,1998-05-08,30.910999,2,0.000,128,18,1998
3,71652,1998-05-08,28.966999,2,1.718,128,18,1998
4,72641,1998-05-08,25.083000,2,0.000,128,18,1998


In [88]:
weekly    = weatherData.drop(columns = {'date'}).groupby(['ZIP','week','quarter','year']).mean().reset_index()
quarterly = weatherData.drop(columns = {'date','week'}).groupby(['ZIP','quarter','year']).mean().reset_index()

# mmonthly   = weatherData.drop(columns = {'date','week'}).groupby(['ZIP','month','quarter','year']).mean().reset_index()

In [52]:
weekly.head()

,ZIP,week,quarter,year,temperature,precipitation,dayOfYear
0,1001,0,1,1981,-7.465167,0.268833,3.5
1,1001,0,1,1982,3.875000,13.926667,3.5
2,1001,0,1,1983,2.708000,2.790833,3.5
3,1001,0,1,1984,1.857833,0.076333,3.5
4,1001,0,1,1985,2.270667,1.525667,3.5


In [89]:
start = time.time()

mediansWeekly    = weekly[['week','ZIP','temperature','precipitation']].groupby(['week','ZIP']).quantile(.5).reset_index()
mediansWeekly.rename(columns = {'temperature': 'tempWeek_50',
                               'precipitation': 'precWeek_50'}, inplace = True)
print(time.time() - start)

ninetyFiveWeekly = weekly[['week','ZIP','temperature','precipitation']].groupby(['week','ZIP']).quantile(.95).reset_index()
ninetyFiveWeekly.rename(columns = {'temperature': 'tempWeek_95',
                               'precipitation': 'precipWeek_95'}, inplace = True)
print(time.time() - start)

weeklyQuartiles = mediansWeekly.merge(ninetyFiveWeekly)



24.835949182510376
47.79756498336792


In [90]:
start = time.time()

mediansQuarterly    = quarterly[['quarter','ZIP','temperature','precipitation']].groupby(['quarter','ZIP']).quantile(.5).reset_index()
mediansQuarterly.rename(columns = {'temperature': 'tempQuarter_50',
                               'precipitation': 'precQuarter_50'}, inplace = True)
print(time.time() - start)

ninetyFiveQuarterly = quarterly[['quarter','ZIP','temperature','precipitation']].groupby(['quarter','ZIP']).quantile(.95).reset_index()
ninetyFiveQuarterly.rename(columns = {'temperature': 'tempQuarter_95',
                               'precipitation': 'precQuarter_95'}, inplace = True)
print(time.time() - start)


quarterlyQuartiles = mediansQuarterly.merge(ninetyFiveQuarterly)



1.1977589130401611
2.3113389015197754


In [92]:
outfile =  '../../data/companyData/weeklyQuartiles_8199.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(weeklyQuartiles, pickle_file)

In [93]:
'''outfile =  '../../data/companyData/monthlyQuartiles_8199.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(monthlyQuartiles, pickle_file)'''

"outfile =  '../../data/companyData/monthlyQuartiles_8199.pkl'\nwith open(outfile, 'wb') as pickle_file:\n    pkl.dump(monthlyQuartiles, pickle_file)"

In [94]:
outfile =  '../../data/companyData/quarterlyQuartiles_8199.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(quarterlyQuartiles, pickle_file)

In [97]:
# del temp
del weekly
# del monthly
del quarterly
gc.collect()

445

In [98]:
del weatherData
gc.collect()

20

# Find Quartiles by Zip

In [2]:
recentDecadeWeather   = dd.read_csv("../../data/companyData/allZipsTemp_0019.csv").drop(columns = {'Unnamed: 0'}).reset_index(drop = True)
precipData = dd.read_csv("../../data/companyData/allZipsPrecip_0019.csv").drop(columns = {'Unnamed: 0'}).reset_index(drop = True)
with ProgressBar():
    precipData    = precipData.repartition(npartitions=225)
    recentDecadeWeather   = recentDecadeWeather.repartition(npartitions=225)


# tempData['precipitation'] = precipData.precipitation
recentDecadeWeather = recentDecadeWeather.merge(precipData).compute(workers = 100) # assign(precipitation=precipData.precipitation)
# recentDecadeWeather.head()

print(recentDecadeWeather.dtypes,sys.getsizeof(recentDecadeWeather)/1e9)

recentDecadeWeather.ZIP           = recentDecadeWeather.ZIP.astype('int32')
recentDecadeWeather.temperature   = recentDecadeWeather.temperature.astype('float32')
recentDecadeWeather.precipitation = recentDecadeWeather.precipitation.astype('float32')


print(recentDecadeWeather.dtypes)

sys.getsizeof(recentDecadeWeather)/1e9

ZIP                int64
date             float64
temperature      float64
quarter            int64
precipitation    float64
dtype: object 11.450499872
ZIP                int32
date             float64
temperature      float32
quarter            int64
precipitation    float32
dtype: object


8.587874912

In [3]:
recentDecadeWeather['date'] = pd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d')
recentDecadeWeather['dayOfYear'] = recentDecadeWeather.date.dt.dayofyear
recentDecadeWeather['week'] = recentDecadeWeather.dayOfYear//7

recentDecadeWeather['year']       = pd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d').dt.year
recentDecadeWeather['quarter']    = pd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d').dt.quarter
# recentDecadeWeather['month']      = dd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d').dt.month

In [5]:
file = open('../../data/companyData/weeklyQuartiles_8199.pkl','rb')
weeklyQuartiles = pkl.load(file)

'''file = open('../../data/companyData/monthlyQuartiles_8199.pkl','rb')
monthlyQuartiles = pkl.load(file)'''

file = open('../../data/companyData/quarterlyQuartiles_8199.pkl','rb')
quarterlyQuartiles = pkl.load(file)

In [6]:
recentDecadeWeather.head()

,ZIP,date,temperature,quarter,precipitation,dayOfYear,week,year
0,52626,2019-08-12,29.313000,3,0.362,224,32,2019
1,53048,2019-08-12,24.110001,3,0.000,224,32,2019
2,53105,2019-08-12,24.587000,3,0.000,224,32,2019
3,54443,2019-08-12,23.110001,3,2.620,224,32,2019
4,55014,2019-08-12,26.896999,3,0.000,224,32,2019


In [7]:
quarterlyQuartiles.head()

,quarter,ZIP,tempQuarter_50,precQuarter_50,tempQuarter_95,precQuarter_95
0,1,1001,4.321811,2.985178,6.527353,4.057065
1,1,1002,3.033165,2.932644,5.151669,3.850850
2,1,1003,3.912275,2.753545,6.091045,3.701070
3,1,1005,2.158374,2.896078,3.962982,3.912775
4,1,1007,3.426022,2.827722,5.537153,3.871607


In [8]:
weeklyQuartiles.head()

,week,ZIP,tempWeek_50,precWeek_50,tempWeek_95,precipWeek_95
0,0,1001,2.270667,2.034667,6.712183,7.039566
1,0,1002,0.961500,2.044333,5.698384,5.735150
2,0,1003,1.839333,1.705667,6.414233,5.837050
3,0,1005,0.582667,2.541833,5.064484,7.345600
4,0,1007,1.576333,2.354167,6.041900,5.984550


In [9]:
weeklyLastDec    = recentDecadeWeather.drop(columns = {'date'}).groupby(['ZIP','week','quarter','year']).mean().reset_index()
quarterlyLastDec = recentDecadeWeather.drop(columns = {'date','week'}).groupby(['ZIP','quarter','year']).mean().reset_index()

In [10]:
del recentDecadeWeather
gc.collect()

67

In [7]:
ZIPs = weeklyQuartiles.ZIP.unique()
len(ZIPs)

32656

In [12]:
print(weeklyLastDec.shape)
weeklyLastDec = weeklyLastDec.merge(weeklyQuartiles)
weeklyLastDec.shape

(35595040, 7)


(35595040, 11)

In [13]:
print(quarterlyLastDec.shape)
quarterlyLastDec = quarterlyLastDec.merge(quarterlyQuartiles)
quarterlyLastDec.shape

(2612480, 6)


(2612480, 10)

In [14]:
quarterlyLastDec.head()

,ZIP,quarter,year,temperature,precipitation,dayOfYear,tempQuarter_50,precQuarter_50,tempQuarter_95,precQuarter_95
0,1001,1,2000,5.348000,2.657242,46.0,4.321811,2.985178,6.527353,4.057065
1,1001,1,2001,3.113778,2.887744,45.5,4.321811,2.985178,6.527353,4.057065
2,1001,1,2002,6.893611,2.303844,45.5,4.321811,2.985178,6.527353,4.057065
3,1001,1,2003,1.947078,2.972100,45.5,4.321811,2.985178,6.527353,4.057065
4,1001,1,2004,2.729110,1.421110,46.0,4.321811,2.985178,6.527353,4.057065


In [31]:
weeklyLastDec['temp_zipWeek50'] = 1*(weeklyLastDec.temperature > weeklyLastDec.tempWeek_50)
weeklyLastDec['temp_zipWeek95'] = 1*(weeklyLastDec.temperature > weeklyLastDec.tempWeek_95)

weeklyLastDec['precip_zipWeek50'] = 1*(weeklyLastDec.precipitation > weeklyLastDec.precWeek_50)
weeklyLastDec['precip_zipWeek95'] = 1*(weeklyLastDec.precipitation > weeklyLastDec.precipWeek_95)

weeklyLastDec.head()

weeklyCounts = weeklyLastDec[['ZIP','quarter','year',
                                    'temp_zipWeek50','temp_zipWeek95',
                                   'precip_zipWeek50','precip_zipWeek95']].\
                                    groupby(['ZIP','quarter','year']).\
                                    sum().reset_index()

weeklyCounts.head()

,ZIP,quarter,year,temp_zipWeek50,temp_zipWeek95,precip_zipWeek50,precip_zipWeek95
0,1001,1,2000,8,2,8,0
1,1001,1,2001,5,0,7,2
2,1001,1,2002,10,3,6,0
3,1001,1,2003,4,1,6,2
4,1001,1,2004,5,1,4,1


In [33]:
weeklyCounts.temp_zipWeek95.describe()

count    2.612480e+06
mean     1.383345e+00
std      1.384789e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      1.400000e+01
Name: temp_zipWeek95, dtype: float64

In [23]:
quarterlyLastDec['temp_zipQuarter50'] = 1*(quarterlyLastDec.temperature > quarterlyLastDec.tempQuarter_50)
quarterlyLastDec['temp_zipQuarter95'] = 1*(quarterlyLastDec.temperature > quarterlyLastDec.tempQuarter_95)

quarterlyLastDec['precip_zipQuarter50'] = 1*(quarterlyLastDec.precipitation > quarterlyLastDec.precQuarter_50)
quarterlyLastDec['precip_zipQuarter95'] = 1*(quarterlyLastDec.precipitation > quarterlyLastDec.precQuarter_95)

quarterlyLastDec.head()

quarterlyCounts = quarterlyLastDec[['ZIP','quarter','year',
                                    'temp_zipQuarter50','temp_zipQuarter95',
                                   'precip_zipQuarter50','precip_zipQuarter95']].\
                                    groupby(['ZIP','quarter','year']).\
                                    sum().reset_index()

quarterlyCounts.head()

,ZIP,quarter,year,temp_zipQuarter50,temp_zipQuarter95,precip_zipQuarter50,precip_zipQuarter95
0,1001,1,2000,1,0,0,0
1,1001,1,2001,0,0,0,0
2,1001,1,2002,1,1,0,0
3,1001,1,2003,0,0,0,0
4,1001,1,2004,0,0,0,0


In [27]:
quarterlyCounts.temp_zipQuarter50.describe()

count    2.612480e+06
mean     6.036582e-01
std      4.891371e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: temp_zipQuarter50, dtype: float64

In [40]:
countDataRevised = quarterlyCounts.merge(weeklyCounts)
countDataRevised.rename(columns = {'ZIP': 'zipcode'}, inplace = True)

In [41]:
print(quarterlyCounts.shape, weeklyCounts.shape, countDataRevised.shape)

(2612480, 7) (2612480, 7) (2612480, 11)


In [42]:
countDataRevised.to_csv("../../data/companyData/weekMonthQuarter_2000to2019_allZips.csv")


Add in the 90+ days so that we're good to go more readily.

In [39]:
quarterlyExtremes = pd.read_csv("../../data/companyData/heat_above90Quarterly.csv").drop(columns = 'Unnamed: 0')
quarterlyExtremes['year']    = quarterlyExtremes.yearQuarter.str.slice(0,4).astype('int64')
quarterlyExtremes['quarter'] = quarterlyExtremes.yearQuarter.str.slice(5,6).astype('int64')

quarterlyExtremes.rename(columns = {'extreme': 'days90Plus',
                                   'heatWave': 'streak90Plus',
                                   'ZIP': 'zipcode'}, 
                        inplace = True)

quarterlyExtremes.drop(columns = 'yearQuarter', inplace = True)

quarterlyExtremes.astype({'zipcode': 'category',
                          'quarter': 'category',
                          'year': 'category'})

,zipcode,days90Plus,streak90Plus,year,quarter
0,1001,0,0,2000,1
1,1001,5,0,2000,2
2,1001,0,0,2000,3
3,1001,0,0,2000,4
4,1001,0,0,2001,1
...,...,...,...,...,...
2612475,99403,0,0,2018,4
2612476,99403,0,0,2019,1
2612477,99403,0,0,2019,2
2612478,99403,17,0,2019,3


In [43]:
countDataRevised = countDataRevised.merge(quarterlyExtremes)

countDataRevised.head()

,zipcode,quarter,year,temp_zipQuarter50,temp_zipQuarter95,precip_zipQuarter50,precip_zipQuarter95,temp_zipWeek50,temp_zipWeek95,precip_zipWeek50,precip_zipWeek95,days90Plus,streak90Plus
0,1001,1,2000,1,0,0,0,8,2,8,0,0,0
1,1001,1,2001,0,0,0,0,5,0,7,2,0,0
2,1001,1,2002,1,1,0,0,10,3,6,0,0,0
3,1001,1,2003,0,0,0,0,4,1,6,2,0,0
4,1001,1,2004,0,0,0,0,5,1,4,1,0,0


In [44]:
countDataRevised.to_csv("../../data/companyData/latestExtremes.csv")
